# Create CIGALE input files

Here we create the photometry files that we use as an input to [CIGALE](https://cigale.lam.fr/). All the code for create this files was develoved by Andrés Ramos.

In [24]:
from datetime import date
import time
import numpy as np
import astropy.units as u

from astroquery.ipac.ned import Ned
from astropy.table import Table, Column
from pathlib import Path

from matplotlib import pyplot as plt

In [17]:
path_raw='../Datos/Raw/'
path_phot_ned = '../Datos/Work/Phot_Tables_NED/' #In this path wi will save the photometry tables for each object
path_phot_cds = '../Datos/Work/Phot_Tables_CDS/'
path_SED = '../Datos/Work/SEDs/'

date_sample = 'Apr_12_2022'
NeIR_obj_t = Table.read(path_raw+'Obj_Ne-IR_'+date_sample+'.vot',format='votable')
NeIR_obj_t.show_in_notebook()

idx,Main_Id,Id_used_NED,RA,DEC,otype_txt,redshift,rvz_type,otypes,cds_url,SMB_sample_index,Ne_IR_info,NED_Ne_IR_info
0,3C 234.0,3C 234.0,150.45634473807,28.7858176354,Sy1,0.18494,z,AGN|G |IR |QSO|Rad|Sy1|X |blu|gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=150.45634473807%2028.7858176354&-c.rs=5,283.0,NeII|NeIII|NeV,NeII|NeIII|NeV
1,M 84,M 84,186.2655972083333,12.886983138888887,Sy2,0.00327,z,* |AGN|G |GiC|GiG|IR |Rad|Sy2|UV |X |smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=186.2655972083333%2012.886983138888887&-c.rs=5,414.0,NeV,NeV
2,2MASS J13000535+1632148,NVSS J130005+163212,195.02228295163,16.53743305218,Sy1,0.07993,z,AGN|G |IR |Rad|Sy1|X,http://vizier.u-strasbg.fr/viz-bin/sed?-c=195.02228295163%2016.53743305218&-c.rs=5,428.0,NeII|NeIII,NeII|NeIII
3,NGC 262,NGC 262,12.196426138730002,31.956956641230004,Sy2,0.015260233939595302,v,* |AGN|Bla|G |GiG|IR |QSO|Rad|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=12.196426138730002%2031.956956641230004&-c.rs=5,484.0,NeII|NeIII|NeV,NeII|NeIII|NeV
4,IRAS 04385-0828,IRAS 04385-0828,70.2290037068,-8.37281297812,Sy2,0.015064,z,* |AGN|G |GiG|IR |Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=70.2290037068%20-8.37281297812&-c.rs=5,541.0,NeII|NeIII|NeV,NeII|NeIII|NeV
5,NGC 4945,NGC 4945,196.3636625,-49.4679,Sy2,0.0017968563040435814,v,AGN|G |GiG|IR |Rad|Sy2|X |cm |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=196.3636625%20-49.4679&-c.rs=5,628.0,NeII|NeIII|NeV,NeII|NeIII|NeV
6,IC 4553,IC 4553,233.73843316666665,23.50322455555555,SyG,0.0184,z,AGN|G |GiG|GiP|IG |IR |LIN|Rad|SyG|X |gam|smm,http://vizier.u-strasbg.fr/viz-bin/sed?-c=233.73843316666665%2023.50322455555555&-c.rs=5,654.0,NeII|NeIII|NeV,NeII|NeIII|NeV
7,NGC 5252,NGC 5252,204.56612821846997,4.542644796,Sy2,0.02294,z,AG?|AGN|EmG|G |GiG|Sy1|Sy2|X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=204.56612821846997%204.542644796&-c.rs=5,773.0,NeII,NeII
8,Mrk 279,Mrk 279,208.26431206837998,69.30816969735,Sy1,0.030601,z,AGN|G |IR |QSO|Sy1|UV |X |gam,http://vizier.u-strasbg.fr/viz-bin/sed?-c=208.26431206837998%2069.30816969735&-c.rs=5,852.0,NeII|NeIII|NeV,NeII|NeIII|NeV
9,UGC 11680,UGC 11680,316.93375,3.875277777777778,Sy2,0.026,z,G |GiP|IG |IR |PaG|Sy2,http://vizier.u-strasbg.fr/viz-bin/sed?-c=316.93375%203.875277777777778&-c.rs=5,1227.0,NeII,NeII


### Create equivalences between NED and CDS tables
Here we create a list for the different names of the bands in NED and CDS. Each row represents the same band.

In [15]:
Equival = [['2-10 keV (XMM)', None, None],
           ['0.5-2 keV (XMM)', None, None],
           ['GALEX:FUV', 'FUV (GALEX)', None],
           ['GALEX:NUV', 'NUV (GALEX)', None],
           ["SDSS:u'", 'SDSS:u', 'u (SDSS) AB'],
           ["SDSS:g'", 'SDSS:g', 'g (SDSS) AB'],
           ["SDSS:r'", 'SDSS:r', 'r (SDSS) AB'],
           ["SDSS:i'", 'SDSS:i', 'i (SDSS) AB'],
           ["SDSS:z'", 'SDSS:z', 'z (SDSS) AB'],
           ['2MASS:J', 'J (2MASS) AB', None],
           ['2MASS:H', 'H (2MASS) AB', None],
           ['2MASS:Ks', 'Ks (2MASS) AB', None],
           ['WISE:W1', 'W1 (WISE)', None],
           [':=3.6um', '3.6 microns (IRAC)', None],
           [':=4.5um', '4.5 microns (IRAC)', None],
           ['WISE:W2', 'W2 (WISE)', None],
           [':=5.8um', '5.8 microns (IRAC)', None],
           [':=8um', '8.0 microns (IRAC)', None],
           ['IRAS:12', '12 microns (IRAS)', None],
           ['WISE:W3', 'W3 (WISE)', None],
           ['WISE:W4', 'W4 (WISE)', None],
           ['Spitzer/MIPS:24', '24 microns (MIPS)', None],
           ['IRAS:25', '25 microns (IRAS)', None],
           ['IRAS:60', '60 microns (IRAS)', None],
           ['Herschel/PACS:70', '70 microns (PACS)', None],
           ['Spitzer/MIPS:70', '70 microns (MIPS)', None],
           ['IRAS:100', '100 microns (IRAS)', None],
           ['Herschel/PACS:100', '100 microns (PACS)', None],
           ['Spitzer/MIPS:160', '160 microns (MIPS)', None],
           ['Herschel/PACS:160', '160 microns (PACS)', None],
           ['Herschel/SPIRE:250', '250 microns (SPIRE)', ':=250um'],
           ['Herschel/SPIRE:350', '350 microns (SPIRE)', ':=350um'],
           ['Herschel/SPIRE:500', '500 microns (SPIRE)', ':=500um'],
           [':=5GHz', ':=6cm', '4.89 GHz (VLA)'],
           [':=1.5GHz', ':=20cm', '1.46 GHz (VLA)'],
           [':=1.4GHz', ':=21cm', '1.4GHz']
           ]

### Transform photometry to CIGALE table

The next class allows us to transform the units of the SEDs in the selected bands.

In [16]:
class TransformCIGALE:
    """Class to plot the SEDs and create a row with the right units"""
    def __init__(self, sed):
        self.sed = sed
        #self.init_plot() # To check the SED
        self.create_row()

    def init_plot(self):
        "Plot the SED"
        plt.errorbar(self.sed['Wave'], self.sed['Flux'], yerr=self.sed['F_er'], fmt='o')
        plt.xlim(1e-6, 1e6)
        plt.loglog()

    def create_row(self):
        """Calculate the flux and error to use for the SED in CIGALE"""
        # Number of filtets/bands and their errors
        self.row = np.full(72, np.nan)
        iband = 0
        for band in Equival:
            crop = self.sed[np.logical_or.reduce([self.sed['Filter'] == band[0],
                                                  self.sed['Filter'] == band[1],
                                                  self.sed['Filter'] == band[2]])]
            flux = np.mean(crop['Flux'])
            std = np.sqrt(np.sum(crop['F_er']**2))/len(crop)
            self.row[iband] = flux*1000  # To send in mJy
            self.row[iband+1] = std*1000  # To send in mJy
            iband += 2

We create a table with all the CIGALE bands/filters and fill it with the information that we get from NED and CDS. In some cases some of the galaxies are not useful because the bands are not found initially. This could take some minutes.

In [18]:
CIGALEFilter = ['xray_box_car_2to10keV', 'xray_box_car_0p5to2keV', 'FUV', 'NUV',
                'u_prime', 'g_prime', 'r_prime', 'i_prime', 'z_prime', 'J_2mass',
                'H_2mass', 'Ks_2mass', 'WISE1', 'IRAC1', 'IRAC2', 'WISE2', 'IRAC3', 'IRAC4',
                'IRAS1', 'WISE3', 'WISE4', 'MIPS1', 'IRAS2', 'IRAS3', 'PACS_blue', 'MIPS2',
                'IRAS4', 'PACS_green', 'MIPS3', 'PACS_red', 'PSW_ext', 'PMW_ext', 'PLW_ext',
                'VLA_C', 'VLA_L', 'HI21cm']
# VLAC is 6cm ,VLAL is 20 cm and HI21cm is 1.4GHz
CIGFilt = [[filt, filt+'_err'] for filt in CIGALEFilter]
CIG = Table(names=np.array(CIGFilt).ravel())
GoodGalaxies = []

for igal, Galaxy in enumerate(NeIR_obj_t['Main_Id']):
    #     figure()
    try:
        sed_file = Table.read(path_SED+Galaxy + '_Phot.txt',
                          format='ascii')
        if len(sed_file) == 0:
            print(Galaxy, igal, 'No useful phot.')
        else:
            GoodGalaxies.append(igal)
            CIG.add_row(TransformCIGALE(sed_file).row)
    except FileNotFoundError:
        print(Galaxy, igal, 'No Phot. ERROR')

/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3417: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/jonhatan/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/tmp/ipykernel_46657/2558554763.py:24: RuntimeWarning: invalid value encountered in double_scalars
  std = np.sqrt(np.sum(crop['F_er']**2))/len(crop)


In [19]:
print('Percentage of galaxies with X-ray data:',
      sum(~np.isnan(CIG[CIG.colnames[1]].data))/len(CIG))

Percentage of galaxies with X-ray data: 0.28133704735376047


We add the identifier (main_id) and the redshift from SIMBAD.

In [25]:
col_name = Column(name='id', data=NeIR_obj_t['Main_Id'][GoodGalaxies])
CIG.add_column(col_name, 0)
col_z = Column(name='redshift', data=NeIR_obj_t['redshift'][GoodGalaxies])
CIG.add_column(col_z, 1)

In [26]:
CIG

id,redshift,xray_box_car_2to10keV,xray_box_car_2to10keV_err,xray_box_car_0p5to2keV,xray_box_car_0p5to2keV_err,FUV,FUV_err,NUV,NUV_err,u_prime,u_prime_err,g_prime,g_prime_err,r_prime,r_prime_err,i_prime,i_prime_err,z_prime,z_prime_err,J_2mass,J_2mass_err,H_2mass,H_2mass_err,Ks_2mass,Ks_2mass_err,WISE1,WISE1_err,IRAC1,IRAC1_err,IRAC2,IRAC2_err,WISE2,WISE2_err,IRAC3,IRAC3_err,IRAC4,IRAC4_err,IRAS1,IRAS1_err,WISE3,WISE3_err,WISE4,WISE4_err,MIPS1,MIPS1_err,IRAS2,IRAS2_err,IRAS3,IRAS3_err,PACS_blue,PACS_blue_err,MIPS2,MIPS2_err,IRAS4,IRAS4_err,PACS_green,PACS_green_err,MIPS3,MIPS3_err,PACS_red,PACS_red_err,PSW_ext,PSW_ext_err,PMW_ext,PMW_ext_err,PLW_ext,PLW_ext_err,VLA_C,VLA_C_err,VLA_L,VLA_L_err,HI21cm,HI21cm_err
object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
3C 234.0,0.18494,nan,nan,nan,nan,0.10159999801544473,0.015652477,0.07938000198919326,0.0065749527,0.12168760283278365,0.004588063170048556,0.19911266922463255,0.0022166697088012214,0.46338666991990374,0.009870157999999999,0.5747533335428064,0.01403923,0.604496665497815,0.016634980329802016,2.0773333342125015,0.61318843,3.3460000685105724,1.1375853,5.038461468827267,0.6604544,11.51666669199864,0.39463524507012804,14.700000000000001,0.025770137756713683,25.4,0.036741393550054686,24.97916678947707,0.8645185790219333,42.7,0.1023783180170489,nan,nan,160.0,28.8,115.22500169277191,3.052321210784712,289.92499858140945,9.236632520552552,274.5,0.3078587338374535,258.0,35.2,231.0,36.900000000000006,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
M 84,0.00327,nan,nan,nan,nan,0.7200916729404949,0.1616562275281175,3.0804883431037724,0.7012790154585334,168.0749986730516,16.887911442660073,276.86499935186777,8.58330425911389,546.3376212349783,24.454584718653074,457.6099968398921,56.28791708861703,721.0112630273215,147.3731,863.4142864140725,627.2094999999999,1214.1266842683156,209.51718,763.1357269627707,168.03016000000002,457.67308149429465,32.201410809852725,412.5,80.67775406728666,175.06666666666666,46.30000003142013,304.27307501664524,28.8284003110999,111.39999999999999,28.400000794480626,105.875,19.01195749471895,178.12500163912776,43.31584441972735,44.0,0.648,36.1,1.23,58.58750036656857,8.180516190215688,184.99999928474426,53.66192586742391,484.91666493813193,77.27167578899156,nan,nan,596.999993165334,47.39989549402087,1002.9166744152707,285.8325629105615,2390.0000351667404,411.0887763635612,586.833343664805,78.71308679278323,4257.249997138977,852.0699480349735,239.83333381017047,27.249816016755506,139.66666730244955,8.79242094253477,125.24999998509884,11.006361529497669,nan,nan,nan,nan,5174.999942736967,714.1433000000001
2MASS J13000535+1632148,0.07993,nan,nan,nan,nan,0.006010000106471125,0.0027,0.024666666528598096,0.005960704,0.21484999888343737,0.016248076889447016,0.8364499924937263,0.018411953325684707,2.232700002787169,0.08780091509073495,3.4850151430212217,0.05192542858582773,5.616999889025465,0.07280109280276387,4.738888832637006,0.6480741,7.277777754805154,1.1301769,11.521249893121421,1.1670904000000002,25.551666749812043,0.9731656247574831,nan,nan,nan,nan,41.49033305126552,1.2555127387884453,nan,nan,nan,nan,nan,nan,90.7500009417534,1.8943277130925162,173.89583517362672,7.663102869566365,148.0,0.11657684161101638,nan,nan,nan,nan,nan,nan,68.9999982714653,7.4,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,11.375000012665986,0.6383572813228967
NGC 262,0.015260233939595302,na